In [4]:
# STEP 1
# Goal: Reading the CSV File, Find ecDNA in the CSV File => list of cycle files name

# Imports
import pandas as pd

# Variables
filePath = "~/Downloads/results-2/"
fileName = "aggregated_results.csv"
suffixOfCycleFileName = "_annotated_cycles.txt"
cycleFileNames = set()

# Filters rows that have the classification 'ecDNA'
dataTable = pd.read_csv(filePath + fileName)
dataTable = dataTable[dataTable['Classification'].notna()]
dataTable = dataTable[dataTable['Classification'].str.contains('ecDNA')]

dataTable = dataTable['Feature ID']

# Makes a list of cycle file names
for id in dataTable:
    index = id.index('amplicon')
    cycleFileNames.add(id[:index+9] + suffixOfCycleFileName)

print(cycleFileNames)

{'HCC827_LUNG_amplicon3_annotated_cycles.txt', 'HDQP1_BREAST_amplicon3_annotated_cycles.txt', 'HCC44_LUNG_amplicon3_annotated_cycles.txt', 'UACC893_BREAST_amplicon6_annotated_cycles.txt', 'OVSAHO_OVARY_amplicon2_annotated_cycles.txt', '5637_URINARY_TRACT_amplicon1_annotated_cycles.txt', 'HCC202_BREAST_amplicon2_annotated_cycles.txt', 'QGP1_PANCREAS_amplicon4_annotated_cycles.txt', 'HCC1599_BREAST_amplicon3_annotated_cycles.txt', 'KNS62_LUNG_amplicon2_annotated_cycles.txt', 'CALU6_LUNG_amplicon1_annotated_cycles.txt', 'CHAGOK1_LUNG_amplicon4_annotated_cycles.txt', 'HCC38_BREAST_amplicon7_annotated_cycles.txt', 'NCIH1435_LUNG_amplicon3_annotated_cycles.txt', 'HT115_LARGE_INTESTINE_amplicon1_annotated_cycles.txt', 'NCIH1694_LUNG_amplicon1_annotated_cycles.txt', 'OVSAHO_OVARY_amplicon8_annotated_cycles.txt', 'RMGI_OVARY_amplicon6_annotated_cycles.txt', 'MFE319_ENDOMETRIUM_amplicon1_annotated_cycles.txt', 'OE19_OESOPHAGUS_amplicon2_annotated_cycles.txt', 'CHP212_AUTONOMIC_GANGLIA_amplicon1_

In [5]:
# STEP 2
# Goal: Open the Feature ID under cycles.txt file, find the cycles in corresponding cycles.txt file 
# => file with segments & only actual cycle content

segment_list = []

for file_name in cycleFileNames:
    sample_name = file_name.split('_')[0]
    path = f'../Downloads/results-2/other_files/ccle_hg38_ac/ccle_hg38_annotated_cycles_files/{file_name}'

    with open(path) as file:

        # skip lines until cycle info 
        file.readline() # skip title 
        line = file.readline()
        while line[0] == 'S':
            line = file.readline()
        
        # find segments for cycles
        segments = set()
        while line:
            sep_line = line.split(';')
            cyclic = sep_line[3]
            cyclic_bool = cyclic.split('=')[1]
            if cyclic_bool == 'True':
                cycle_segments = sep_line[-1].split('=')[1]
                cycle_segments = cycle_segments.split(',')
                for i in range(len(cycle_segments)):
                    cycle_segments[i] = cycle_segments[i].strip('+-\n')
                for seg in cycle_segments:
                    segments.add(seg)
            line = file.readline()

        # identify chr positions for cyclic segments 
        file.seek(0)
        file.readline() # skip title
        line = file.readline()
        while line[0] == 'S':
            sep_line = line.strip().split('\t')
            new_line = sample_name + ',' + sep_line[2] + ',' + sep_line[3] + ',' + sep_line[4]
            if (sep_line[1] in segments) and (new_line not in segment_list):
                segment_list.append(new_line)
            line = file.readline()   

# remove repetitions
df = pd.DataFrame(columns=['sample', 'chr', 'start_pos', 'end_pos'])
for i in segment_list:
    sep_line = i.split(',')
    df.loc[len(df.index)] = [sep_line[0], sep_line[1], sep_line[2], sep_line[3]]
name = f'ccle-outputs/cyclic-segments.tsv'
df.to_csv(name, sep='\t', index=False)


In [8]:
# STEP 3
# Goal: Check each segment to see if they are less than 1000 base pairs => file with chromosome coordinates (no repetitions)

samples = []
chromnum = []
startingcoord = []
endingcoord = []

with open ('ccle-outputs/cyclic-segments.tsv') as file:
    file.readline() # skip header
    for line in file:
        l = line.strip()
        t = l.split('\t')
        samples.append(t[0])
        chromnum.append(t[1])
        startingcoord.append(int(t[2]))
        endingcoord.append(int(t[3]))

df = pd.DataFrame(columns=['sample', 'chr', 'start_pos', 'end_pos'])
for i in range(len(startingcoord)):
    if ((endingcoord[i]-startingcoord[i]) < 1000):
        df.loc[len(df.index)] = [samples[i], chromnum[i], startingcoord[i], endingcoord[i]]
name = f'ccle-outputs/slivers.tsv'
df.to_csv(name, sep='\t', index=False, header=False)

In [10]:
# STEP 4
# Goal: Extract sequences from hg38 ref genome 

# this function will return DNA sequence based on the marker file
# marker file format example: Chr7 1 9
def findSliverSeq(marker_file):
    markers = open(marker_file).readlines()
    result_seqs = []

    # record information about sliver seq to write to csv file later 
    samples = []
    chr_header = []
    start_pos = []
    end_pos = []

    # load genome
    chr2seq = pd.read_pickle('hg38.fa.chr2seq.pydict.pickle')

    for marker in markers:
        marker_list = marker.strip().split('\t')

        # record info on input file to output to df later
        samples.append(marker_list[0])
        chr_header.append(marker_list[1])
        start_pos.append(int(marker_list[2]))
        end_pos.append(int(marker_list[3]))

        # find actual sliver sequences 
        chr_num = marker_list[1]
        start = int(marker_list[2])
        end = int(marker_list[3])

        # extract sequence
        seq = chr2seq[chr_num][start:end+1]
        result_seqs.append(seq)

    # append all result sequences to a text file line by line 
    # format like: 
    # ACTCTCTC
    # AACTTTTCCCC
    output = ''
    for sliver_seq in result_seqs:
        output = output + sliver_seq + '\n'
    
    # open result file and write to result file with all sliver seuqneces 
    name1 = f'ccle-outputs/sliver-sequences.txt'
    with open(name1, 'w') as result_file:
        result_file.write(output)
    result_file.close() 

    # open a csv file for storing all info about sliver sequence found 
    # cotains chromosome pos and actual sliver sequnce 
    df = pd.DataFrame()
    df['Sample'] = samples
    df['Chromosome Position'] = chr_header
    df['Start Position'] = start_pos
    df['End Position'] = end_pos
    df['Sequence'] = result_seqs
    name2 = f'ccle-outputs/final-slivers.tsv'
    df.to_csv(name2, sep='\t', index=False)

# run function on data 
findSliverSeq('ccle-outputs/slivers.tsv')